In [1]:
pip install --upgrade --force-reinstall chromedriver-binary-auto

  Using cached chromedriver_binary_auto-0.1.1-py3-none-any.whl
  Attempting uninstall: chromedriver-binary-auto
    Found existing installation: chromedriver-binary-auto 0.1.1
    Uninstalling chromedriver-binary-auto-0.1.1:
      Successfully uninstalled chromedriver-binary-auto-0.1.1
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\xiaob\\anaconda3\\envs\\webscraping\\Lib\\site-packages\\~~romedriver_binary\\chromedriver.exe'
Consider using the `--user` option or check the permissions.



In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_binary
import time
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.support.ui import Select
import json
import os
from pathlib import Path

In [3]:
driver = webdriver.Chrome()

### Set deafult scraping website and saving path

In [4]:
deafult_url = "https://www.wenku8.net/book/{0}.htm"
deafult_save_path = ""
# 1 - 3055
novel_list = [deafult_url.format(x) for x in range(1,3056)]

### Login to website

In [5]:
#login
driver.get("https://www.wenku8.net/book/1.htm")
btn_login = driver.find_element_by_xpath('/html/body/div[1]/div[1]/a[1]')
btn_login.click()

user = driver.find_element_by_name('username').send_keys("")
pwd = driver.find_element_by_name('password').send_keys("")
select = Select(driver.find_element_by_name('usecookie'))
select.select_by_value('86400')
submit = driver.find_element_by_name('submit').click()


### Function to parse each individual novel subwesite

In [6]:
def parse_website():
    obj = {}
    soup = BeautifulSoup(driver.page_source,"html.parser")
    trs = soup.find_all('tr')
    title = [tr.text.strip() for tr in trs if "推一下" in tr.text][0].split('[')[0].strip()
    obj['小说名字'] = title
    tr = [tr.text.strip() for tr in trs if "文库分类" in tr.text]
    info = [i.split('\n') for i in tr][0]
    for i in info:
        obj[i.split('：')[0].strip()] = i.split('：')[1].strip()
    return obj

### Download function

In [7]:
def download_novel(category,name):
    btn = WebDriverWait(driver,3).until(EC.presence_of_element_located((By.XPATH,"//*[contains(text(), 'TXT简繁全本')]"))).click()
    soup = BeautifulSoup(driver.page_source,"html.parser")
    tds = soup.find_all('td')
    size_title = "文件大小"
    size = [td.text.strip() for td in tds if "(G版)/" in td.text][0].split('(G')[0].split('K')
    #download file
    dbtn = WebDriverWait(driver,3).until(EC.presence_of_element_located((By.XPATH,"//td[contains(text(), '简体(G)')]//a")))
    txt_url = dbtn.get_attribute('href')
    driver.get(txt_url)
    content = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'pre')))
    Path(deafult_save_path+category).mkdir(parents=True, exist_ok=True)
    with open(deafult_save_path+category+"\\"+name+".txt",'w', encoding='UTF-8') as f:
        f.write(content.text)
    return {size_title:size[0],'大小单位':'k','文件位置':deafult_save_path+category+"\\"+name+".txt"}
    
#download_novel('Fami通文库',"文学少女")

### scraping start

In [8]:
alldata = []
#scrape the website
count = 1
for n in novel_list:
    try:
        driver.get(n)
        obj = parse_website()
        obj['url'] = n
        try:
            obj.update(download_novel(obj['文库分类'],obj["小说名字"]))
            obj['能否下载'] = ['可以']
        except Exception as e:
            obj['能否下载'] = ['不可以']
            obj["文件大小"] = ['无']
            obj['大小单位'] = ['无']
            obj['文件位置'] = ['无']
            print(obj['小说名字'] + "无法下载")
        alldata.append(obj)
        print('完成: {0} - '.format(count) + obj['小说名字'] + "  完成百分之: %.2f" % (count/30.55) + "%")
    except Exception as e:
        print("该文章不存在")
    count += 1

完成: 1 - 文学少女  完成百分之: 0.03%
TIGER×DRAGON！(龙与虎)无法下载
完成: 2 - TIGER×DRAGON！(龙与虎)  完成百分之: 0.07%
魔法禁书目录无法下载
完成: 3 - 魔法禁书目录  完成百分之: 0.10%
完成: 4 - 笨蛋，测验，召唤兽  完成百分之: 0.13%
完成: 5 - 狼与香辛料(狼与辛香料)  完成百分之: 0.16%
完成: 6 - 化物语(物语系列一)  完成百分之: 0.20%
完成: 7 - 碧阳学园学生会议事录(学生会的一己之见)  完成百分之: 0.23%
全金属狂潮(惊爆危机)无法下载
完成: 8 - 全金属狂潮(惊爆危机)  完成百分之: 0.26%
完成: 9 - 说谎的男孩与坏掉的女孩  完成百分之: 0.29%
完成: 10 - 对某飞行员的追忆  完成百分之: 0.33%
黄昏色的咏使无法下载
完成: 11 - 黄昏色的咏使  完成百分之: 0.36%
奇诺之旅无法下载
完成: 12 - 奇诺之旅  完成百分之: 0.39%
完成: 13 - 零之使魔  完成百分之: 0.43%
灼眼的夏娜无法下载
完成: 14 - 灼眼的夏娜  完成百分之: 0.46%
该文章不存在
完成: 16 - 神的记事本  完成百分之: 0.52%
完成: 17 - 戏言系列  完成百分之: 0.56%
完成: 18 - 虫之歌  完成百分之: 0.59%
完成: 19 - 艾莉森  完成百分之: 0.62%
完成: 20 - 伯爵与妖精  完成百分之: 0.65%
该文章不存在
完成: 22 - 初恋魔法电击  完成百分之: 0.72%
该文章不存在
完成: 24 - 电波系彼女  完成百分之: 0.79%
完成: 25 - 恶魔少女  完成百分之: 0.82%
完成: 26 - 终焉的年代记  完成百分之: 0.85%
完成: 27 - 最终兵器彼女  完成百分之: 0.88%
完成: 28 - 月姬  完成百分之: 0.92%
完成: 29 - 镇魂系列  完成百分之: 0.95%
完成: 30 - 最后大魔王  完成百分之: 0.98%
完成: 31 - 夜明前的琉璃色  完成百分之: 1.01%
完成: 32 - 月兔公主  完成百分之: 1.05%
完成: 33 - 月兔升空时 

完成: 282 - 南方之岛·梦境之中  完成百分之: 9.23%
该文章不存在
完成: 284 - 鸟笼庄的房客今日也慵懒  完成百分之: 9.30%
完成: 285 - 幽灵少女与科学少年  完成百分之: 9.33%
完成: 286 - 美丽人生  完成百分之: 9.36%
完成: 287 - 明天好天气  完成百分之: 9.39%
完成: 288 - 梦幻都市  完成百分之: 9.43%
完成: 289 - 魔女的红线  完成百分之: 9.46%
完成: 290 - 魔界转生  完成百分之: 9.49%
完成: 291 - 某一天，炸弹从天而降  完成百分之: 9.53%
MAMA无法下载
完成: 292 - MAMA  完成百分之: 9.56%
完成: 293 - 七人武器店  完成百分之: 9.59%
完成: 294 - 柳生忍法帖  完成百分之: 9.62%
完成: 295 - 柳生忍法帖(第二部)  完成百分之: 9.66%
完成: 296 - 龙眠  完成百分之: 9.69%
该文章不存在
完成: 298 - 卡莲坂高中可爱广播社  完成百分之: 9.75%
完成: 299 - 空想科学读本  完成百分之: 9.79%
完成: 300 - 灵感少女  完成百分之: 9.82%
完成: 301 - 简单易懂的现代魔法  完成百分之: 9.85%
该文章不存在
该文章不存在
完成: 304 - 混沌军团  完成百分之: 9.95%
完成: 305 - 后院的神明  完成百分之: 9.98%
完成: 306 - DAS SCHWARZE HERZ 黑色心脏  完成百分之: 10.02%
完成: 307 - 鬼哭街  完成百分之: 10.05%
完成: 308 - 怪物猎人  完成百分之: 10.08%
完成: 309 - 骨王  完成百分之: 10.11%
完成: 310 - 狗牙灭绝之刃(狗牙绝剑)  完成百分之: 10.15%
完成: 311 - 姑获鸟之夏  完成百分之: 10.18%
完成: 312 - 银之镇魂歌  完成百分之: 10.21%
该文章不存在
完成: 314 - 风之王国  完成百分之: 10.28%
完成: 315 - 歌剧系列  完成百分之: 10.31%
完成: 316 - 光明之风×光明之泪  完成百分之: 10.34%
完

完成: 535 - 狂野历险 ADVANCED 3rd  完成百分之: 17.51%
完成: 536 - 狂野历险 Alter code:F  完成百分之: 17.55%
完成: 537 - 狂野历险 the Vth Vanguard  完成百分之: 17.58%
完成: 538 - 生化危机 空白玫瑰  完成百分之: 17.61%
完成: 539 - JOJO的奇妙冒险  完成百分之: 17.64%
完成: 540 - 秋之回忆2  完成百分之: 17.68%
完成: 541 - 秋之回忆 从今以后  完成百分之: 17.71%
完成: 542 - 萌单  完成百分之: 17.74%
完成: 543 - 传颂之物  完成百分之: 17.77%
该文章不存在
该文章不存在
完成: 546 - 圣斗士官方小说  完成百分之: 17.87%
完成: 547 - 乙一作品集  完成百分之: 17.91%
完成: 548 - 灼热的龙骑兵  完成百分之: 17.94%
完成: 549 - .hack／／CELL  完成百分之: 17.97%
完成: 550 - .hack／／G.U.  完成百分之: 18.00%
完成: 551 - .hack／／ANOTHER BIRTH  完成百分之: 18.04%
完成: 552 - 地狱少女  完成百分之: 18.07%
完成: 553 - 克蕾利亚传说  完成百分之: 18.10%
完成: 554 - 天元突破 红莲之眼  完成百分之: 18.13%
完成: 555 - 罗密欧的灾难  完成百分之: 18.17%
完成: 556 - 恋姬无双外传 紫电一闪!!华蝶假面  完成百分之: 18.20%
完成: 557 - 重回那天  完成百分之: 18.23%
完成: 558 - 黏黏妖美少女  完成百分之: 18.27%
完成: 559 - 秋之回忆 从今以后Again  完成百分之: 18.30%
完成: 560 - 护花之龙的叙情诗  完成百分之: 18.33%
完成: 561 - 你是主我是仆  完成百分之: 18.36%
完成: 562 - 翼年代记  完成百分之: 18.40%
完成: 563 - 死神候补生  完成百分之: 18.43%
完成: 564 - 初恋限定  完成百分之: 18.46%
完成: 565 - 大剑

完成: 782 - 机动战士高达W 左手持镰，右手拥你  完成百分之: 25.60%
完成: 783 - 机动战士高达W  完成百分之: 25.63%
完成: 784 - 机动战士高达 逆袭的夏亚  完成百分之: 25.66%
完成: 785 - 魔法米克斯  完成百分之: 25.70%
完成: 786 - 完美小姐进化论  完成百分之: 25.73%
完成: 787 - 4TEEN(十四岁)  完成百分之: 25.76%
完成: 788 - 异度传说  完成百分之: 25.79%
完成: 789 - 野猪大改造  完成百分之: 25.83%
完成: 790 - 无人岛物语  完成百分之: 25.86%
完成: 791 - 华莱士人鱼  完成百分之: 25.89%
完成: 792 - 人间之剑  完成百分之: 25.92%
完成: 793 - 前线任务4  完成百分之: 25.96%
完成: 794 - 黑与蓝  完成百分之: 25.99%
该文章不存在
该文章不存在
完成: 797 - 世界在你不知道的地方运转  完成百分之: 26.09%
完成: 798 - 在遥远彼方的小千  完成百分之: 26.12%
完成: 799 - 倘若我在彼岸  完成百分之: 26.15%
完成: 800 - 我的男人  完成百分之: 26.19%
完成: 801 - 守护者系列  完成百分之: 26.22%
完成: 802 - 断章格林童话  完成百分之: 26.25%
完成: 803 - 他人事  完成百分之: 26.28%
完成: 804 - 最后开的花  完成百分之: 26.32%
完成: 805 - 完美无缺的名侦探  完成百分之: 26.35%
完成: 806 - 红线 绊之记忆  完成百分之: 26.38%
完成: 807 - Summer Days(日在夏天)  完成百分之: 26.42%
完成: 808 - 满月之夜白鲸现  完成百分之: 26.45%
完成: 809 - 新古灵精怪(新橙路)  完成百分之: 26.48%
完成: 810 - 零之使魔外传系列  完成百分之: 26.51%
该文章不存在
完成: 812 - 圆环少女circlet girl  完成百分之: 26.58%
完成: 813 - 匣中的失乐  完成百分之: 26.61%
完成: 814 -

完成: 1017 - BLACK BIRD 黑鸟恋人  完成百分之: 33.29%
完成: 1018 - 幻色江户历  完成百分之: 33.32%
完成: 1019 - 乱☆恋 我有16个未婚妻！？  完成百分之: 33.36%
完成: 1020 - 384403km 绑架你到月球  完成百分之: 33.39%
完成: 1021 - 侦探歌剧MilkyHomes～overture～  完成百分之: 33.42%
完成: 1022 - Tales of Eternia 永恒传奇 永远的阶梯(永恒传说)  完成百分之: 33.45%
完成: 1023 - 降临于我与她之夜  完成百分之: 33.49%
完成: 1024 - 魔弹之王与战姬  完成百分之: 33.52%
完成: 1025 - 境界线上的地平线  完成百分之: 33.55%
完成: 1026 - 主人和女仆大人  完成百分之: 33.58%
完成: 1027 - 机动战士高达0079外传 殖民卫星坠落之地…  完成百分之: 33.62%
完成: 1028 - 花X华  完成百分之: 33.65%
完成: 1029 - 有顶天家族  完成百分之: 33.68%
完成: 1030 - 指引之星  完成百分之: 33.72%
完成: 1031 - 蔚蓝的萨利菲拉  完成百分之: 33.75%
完成: 1032 - 魔术士奥芬的无赖之旅(魔术士欧菲流浪之旅)  完成百分之: 33.78%
神不在的星期天无法下载
完成: 1033 - 神不在的星期天  完成百分之: 33.81%
完成: 1034 - 恶魔高校DxD(High School DxD)  完成百分之: 33.85%
完成: 1035 - 突然成为骑士  完成百分之: 33.88%
完成: 1036 - 月光  完成百分之: 33.91%
完成: 1037 - LOVE啾～！  完成百分之: 33.94%
完成: 1038 - 开膛手杰克的百年孤寂  完成百分之: 33.98%
完成: 1039 - 假面骑士Faiz 异形之花  完成百分之: 34.01%
完成: 1040 - 约定之柱，落日的女王  完成百分之: 34.04%
完成: 1041 - 兔子强尼  完成百分之: 34.08%
完成: 1042 - 断罪的EXCEED  完成百分之: 34.

...

完成: 2071 - 山羊岛的蓝色奇迹  完成百分之: 67.79%
完成: 2072 - 超饱和的次代勇者～失去勇者资格的少年～  完成百分之: 67.82%
完成: 2073 - CHAOS；CHILD 某情弱的记录  完成百分之: 67.86%
完成: 2074 - 在监狱学校当看门守卫  完成百分之: 67.89%
完成: 2075 - 游泳社的女生比率太高，我输给诱惑了  完成百分之: 67.92%
完成: 2076 - 我被召唤到魔界成为家庭教师！？  完成百分之: 67.95%
完成: 2077 - 深夜烘焙坊  完成百分之: 67.99%
完成: 2078 - 异世界拷问姬  完成百分之: 68.02%
完成: 2079 - 这才是神父的真面目  完成百分之: 68.05%
完成: 2080 - 平凡职业造就世界最强  完成百分之: 68.09%
完成: 2081 - 折断的圣剑与帝冠剑姬  完成百分之: 68.12%
完成: 2082 - GAMERS电玩咖！  完成百分之: 68.15%
完成: 2083 - Y的纹章师  完成百分之: 68.18%
完成: 2084 - 哥布林杀手  完成百分之: 68.22%
完成: 2085 - 反恋主义同盟！  完成百分之: 68.25%
完成: 2086 - Let’s Party来组队吧！  完成百分之: 68.28%
完成: 2087 - 因为被卷入赌命游戏，所以顺势把讨厌的人杀了  完成百分之: 68.31%
完成: 2088 - 神话传说英雄的异世界奇谭  完成百分之: 68.35%
完成: 2089 - 我家有个地下城  完成百分之: 68.38%
完成: 2090 - WORLD TEACHER 异世界式教育特务  完成百分之: 68.41%
完成: 2091 - 你与时钟系列  完成百分之: 68.45%
完成: 2092 - 审稿男孩和投稿女孩  完成百分之: 68.48%
完成: 2093 - 封神演戏  完成百分之: 68.51%
完成: 2094 - 动手建造迷宫吧  完成百分之: 68.54%
完成: 2095 - 瓦尔哈拉的晚餐(瓦尔哈拉的晚饭)  完成百分之: 68.58%
完成: 2096 - 剪刀·石头·布  完成百分之: 68.61%
完成: 2097 - 幻兽调查员 

完成: 2279 - 苦甜危机！巧克力大骚动  完成百分之: 74.60%
完成: 2280 - 公爵千金的本领  完成百分之: 74.63%
完成: 2281 - 复活后变成Lv1的朕为了活下去只好追求英雄少女了  完成百分之: 74.66%
完成: 2282 - 记忆使者  完成百分之: 74.70%
完成: 2283 - 重生勇者面露冷笑，步上复仇之路  完成百分之: 74.73%
完成: 2284 - 关于人类是最强种族这件事  完成百分之: 74.76%
完成: 2285 - 我喜欢的妹妹不是妹妹(我喜欢的是妹妹但不是妹妹)  完成百分之: 74.80%
完成: 2286 - 灵感少女在盒中  完成百分之: 74.83%
完成: 2287 - 旅团粉碎机的那女孩如果被我已读无视了会露出什么表情呢  完成百分之: 74.86%
完成: 2288 - 魔王悬疑推理剧场 美女勇者遇害的真相  完成百分之: 74.89%
完成: 2289 - 家里蹲的弟弟  完成百分之: 74.93%
完成: 2290 - 这是你与我的最后战场，或是开创世界的圣战(你与我最后的战场，亦或是世界起始的圣战)  完成百分之: 74.96%
完成: 2291 - 少女妄想中  完成百分之: 74.99%
完成: 2292 - 就让我们开演吧。  完成百分之: 75.02%
完成: 2293 - 血翼王亡命谭  完成百分之: 75.06%
完成: 2294 - 物语系列 Off Season(物语系列十三)  完成百分之: 75.09%
完成: 2295 - 世界什么的怎样都好─Qualidea Code─  完成百分之: 75.12%
完成: 2296 - 剑风传奇之火龙骑士  完成百分之: 75.16%
完成: 2297 - 叛逆的无畏少女  完成百分之: 75.19%
完成: 2298 - 掌心之恋与世界的王女们  完成百分之: 75.22%
完成: 2299 - S与S的无赖同盟  完成百分之: 75.25%
完成: 2300 - 虹色异星人  完成百分之: 75.29%
完成: 2301 - 美少女乃求斩之道  完成百分之: 75.32%
完成: 2302 - 境域的伟大秘法(境域的Ars Magna)  完成百分之: 75.35%
完成: 2303 - 目标是与美少女作家一起

完成: 2479 - 救世主的命题  完成百分之: 81.15%
完成: 2480 - 怕痛的我，把防御力点满就对了  完成百分之: 81.18%
完成: 2481 - 在大国开外挂，轻松征服异世界！  完成百分之: 81.21%
完成: 2482 - 终将成为你 关于佐伯沙弥香(终将成为你 佐伯沙弥香的追忆)  完成百分之: 81.24%
完成: 2483 - 天才王子的赤字国家重生术(天才王子的赤字国家振兴术)  完成百分之: 81.28%
完成: 2484 - 精灵、比基尼与机关枪！  完成百分之: 81.31%
完成: 2485 - 替身新娘的贵族生活  完成百分之: 81.34%
完成: 2486 - 月与莱卡与吸血公主  完成百分之: 81.37%
完成: 2487 - 恋爱至上都市的双骑士  完成百分之: 81.41%
完成: 2488 - 口是心非的冰室同学 从好感度100％开始的毒舌女子追求法  完成百分之: 81.44%
完成: 2489 - 把洗发精放在喜欢与讨厌之间  完成百分之: 81.47%
完成: 2490 - 让人想丢辞呈的魔法联盟  完成百分之: 81.51%
完成: 2491 - 天空之上的永恒约定  完成百分之: 81.54%
完成: 2492 - 在世界终焉的花园  完成百分之: 81.57%
完成: 2493 - 老师的新娘是16岁的合法萝莉？  完成百分之: 81.60%
完成: 2494 - 废柴勇者下克上  完成百分之: 81.64%
完成: 2495 - 交叉连结  完成百分之: 81.67%
完成: 2496 - 幸会，食人鬼。  完成百分之: 81.70%
完成: 2497 - 我的快转恋爱喜剧  完成百分之: 81.73%
完成: 2498 - 听见未知的记忆  完成百分之: 81.77%
完成: 2499 - 恋爱必胜女神！  完成百分之: 81.80%
完成: 2500 - 只有我能进入的隐藏迷宫～低调锻炼化身世界最强～  完成百分之: 81.83%
完成: 2501 - 被学生胁迫这事儿是犯罪吗？  完成百分之: 81.87%
完成: 2502 - 舰队Collection 瑞之海，凤之空  完成百分之: 81.90%
完成: 2503 - 透过机器人与你相恋  完成百分之: 81.93%
完成: 250

完成: 2682 - 零之大贤者  完成百分之: 87.79%
完成: 2683 - 我的老婆是兔耳少女！？  完成百分之: 87.82%
完成: 2684 - Thunderbolt Fantasy 东离剑游纪 外传  完成百分之: 87.86%
完成: 2685 - 明日的世界星光闪耀  完成百分之: 87.89%
完成: 2686 - 驱魔学园的叛教者  完成百分之: 87.92%
完成: 2687 - 关于学园的女神大人不知为何一直赖在我家这档事  完成百分之: 87.95%
完成: 2688 - 身为没朋友的死宅的我，不知不觉间就被学院内屈指可数的美少女们包围了，成为了人们口中的现充  完成百分之: 87.99%
完成: 2689 - 冰川双子的生活  完成百分之: 88.02%
完成: 2690 - 最强的魔导士，膝盖中了一箭之后成为乡下的卫兵  完成百分之: 88.05%
完成: 2691 - Owa Rande！梦魔召唤！  完成百分之: 88.09%
完成: 2692 - 谎言游戏(Liar·Liar)  完成百分之: 88.12%
完成: 2693 - 虚构推理  完成百分之: 88.15%
完成: 2694 - 又做了，相同的梦  完成百分之: 88.18%
完成: 2695 - 公主殿下貌似大发雷霆  完成百分之: 88.22%
完成: 2696 - 优雅贵族的休假指南  完成百分之: 88.25%
完成: 2697 - 黑锁姬芙洛莉卡  完成百分之: 88.28%
完成: 2698 - 苍之彼方的四重奏–Fly me to your sky !–  完成百分之: 88.31%
完成: 2699 - 十二大战  完成百分之: 88.35%
完成: 2700 - 侦探已死(侦探已经死了)  完成百分之: 88.38%
完成: 2701 - Steins；Gate命运石之门(剧场版小说)  完成百分之: 88.41%
完成: 2702 - 光明之风 公主们的冒险  完成百分之: 88.45%
完成: 2703 - True Tears 真实之泪  完成百分之: 88.48%
完成: 2704 - TARI TARI~既是萌芽 又是照耀 不时歌唱~  完成百分之: 88.51%
完成: 2705 - 让爱撒娇的大姐姐教官养我，是不是太超过了  完成

完成: 2876 - 怎样都行人家想要回家啦  完成百分之: 94.14%
完成: 2877 - 结与书  完成百分之: 94.17%
完成: 2878 - 生若冬花的你  完成百分之: 94.21%
完成: 2879 - 渴求游戏之神  完成百分之: 94.24%
完成: 2880 - 废材皇子们的皇位争让战～输到最后的一定是我！～  完成百分之: 94.27%
完成: 2881 - 七人的魔剑姬和零之骑士团  完成百分之: 94.30%
完成: 2882 - 与电波系女神的同居日常  完成百分之: 94.34%
完成: 2883 - 义妹生活  完成百分之: 94.37%
完成: 2884 - 超强技能『召唤厨房』！  完成百分之: 94.40%
完成: 2885 - 面对摆出姐姐架子的初恋对象、我是绝对不会屈服的！  完成百分之: 94.44%
完成: 2886 - 妖世刃吊华 吾爱之地 而今安在  完成百分之: 94.47%
完成: 2887 - 魔法禁书目录SS 生物黑客篇  完成百分之: 94.50%
完成: 2888 - 转生王子想懒散度日  完成百分之: 94.53%
完成: 2889 - 从圣女职位被开除后 不知为何幼女化成为了魔王的宠物  完成百分之: 94.57%
完成: 2890 - 请把我当成妹妹，给我超越女友的爱。  完成百分之: 94.60%
完成: 2891 - FGO Mystery  完成百分之: 94.63%
完成: 2892 - 转生王子的英雄谭  完成百分之: 94.66%
完成: 2893 - 0装备栏的最强剑士 但是、诅咒装备(可爱)的话能随意装9999个(我的诅咒装备不可能这么可爱)  完成百分之: 94.70%
完成: 2894 - DELIVERY ROOM  完成百分之: 94.73%
完成: 2895 - 声优广播的幕前幕后(声优广播的台前幕后)  完成百分之: 94.76%
完成: 2896 - 将放言说女生之间不可能的女孩子，在百日之内彻底攻陷的百合故事  完成百分之: 94.80%
完成: 2897 - 冷淡的佐藤同学只对我撒娇  完成百分之: 94.83%
完成: 2898 - 反派千金转职成超级兄控  完成百分之: 94.86%
完成: 2899 - 机动魔装骑  完成百分之: 94.89%
完成: 

In [9]:
print(len(alldata))

2979


### save to csv/excel for future analysis

In [11]:
df = pd.DataFrame(alldata)

In [12]:
df.to_csv('wenku8.csv', index=False)

In [14]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [16]:
df.to_excel('wenku8_raw.xlsx', index=False)